In [2]:
!pip install youtube-transcript-api transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 33.4 MB/s eta 0:00:00


In [3]:
import torch
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [4]:
def get_video_id(url_link):
    video_id_and_timestamp = url_link.split('watch?v=')[-1]
    return video_id_and_timestamp.split('&')[0]

In [5]:
# Initialize Falcon model for summarization
model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [6]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [7]:
# 1. Get transcript
video_link = input('Enter a YouTube link: ')
video_id = get_video_id(video_link)
print("video_id: " + video_id)

Enter a YouTube link: https://www.youtube.com/watch?v=nak-UbbpxnU&list=WL
video_id: nak-UbbpxnU


In [12]:
try:
    ytt_api = YouTubeTranscriptApi()
    fetched_transcript = ytt_api.fetch(video_id)

    text = ""
    for snippet in fetched_transcript:
        text += snippet.text + " "
    text = text[:-1]

    print(f"Transcript length: {len(text)} characters")

    # 2. Generate summary using Falcon
    print("\n⏳ Generating summary with Falcon...")

    # Create a prompt for summarization
    summary_prompt = f"""Please provide a comprehensive summary of the following text:
                        {text[:500]}  # Using first 500 chars to avoid token limits
                        Summary:"""

    # Generate summary
    summary_result = generator(
        summary_prompt,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        num_return_sequences=1,
        return_full_text=False,
        pad_token_id=tokenizer.pad_token_id,
        use_cache=False,
    )

    full_summary = summary_result[0]['generated_text'].strip()

    print("\n Summary:\n")
    print(full_summary)

    # 3. Q&A using Falcon
    print("\n Ask me anything about the video! Type 'quit' to exit.\n")

    while True:
        question = input("Your question: ")
        if question.lower() == 'quit':
            print("Goodbye!")
            break

        # Create Q&A prompt for Falcon
        qa_prompt = f"""Based on the following context, answer the question:
                        Context: {text[:500]}  # Using first 2000 chars for context
                        Question: {question}
                        Answer:"""

        try:
            # Generate answer
            answer_result = generator(
                qa_prompt,
                max_new_tokens=100,
                do_sample=True,
                temperature=0.5,
                top_k=30,
                num_return_sequences=1,
                return_full_text=False,
                pad_token_id=tokenizer.pad_token_id,
                use_cache=False,
            )

            answer = answer_result[0]['generated_text'].strip()
            print("Answer:", answer)

        except Exception as e:
            print(f"Error generating answer: {e}")

except Exception as e:
    print(f"Error fetching transcript: {e}")

Transcript length: 38855 characters

⏳ Generating summary with Falcon...

 Summary:

To land a software engineering internship in 2024, you need to follow these steps: 1. Determine your skills and interests in software engineering. 2. Research companies and opportunities that align with your goals. 3. Prepare a strong resume and cover letter. 4. Network with professionals in the field. 5. Apply to internships that align with your goals. 6. Attend job fairs and events to meet potential employers. 7. Prepare for interviews by practicing common interview questions

 Ask me anything about the video! Type 'quit' to exit.

Your question: How do I meet potential employers?
Answer: Attend career fairs, network with professionals on LinkedIn, and reach out to your personal network for potential opportunities.
Your question: What is the universe?
Answer: The universe is everything that exists, including physical objects and living beings.


KeyboardInterrupt: Interrupted by user